In [ ]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

In [ ]:
import json
import requests
import subprocess
import time
import uuid

from google.adk.agents import LlmAgent
from google.adk.agents.remote_a2a_agent import (
    RemoteA2aAgent,
    AGENT_CARD_WELL_KNOWN_PATH,
)

from google.adk.a2a.utils.agent_to_a2a import to_a2a
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

# Hide additional warnings in the notebook
import warnings

warnings.filterwarnings("ignore")

print("✅ ADK components imported successfully.")

In [ ]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

# Focus Filter - Intelligent Notification Filtering Agent

This notebook implements an AI agent that intelligently filters and manages notifications by:
- Classifying notifications as urgent, irrelevant, or less urgent
- Taking appropriate actions (pass through, block, or store in memory)
- Learning from patterns to improve over time

## Contest Track: Concierge
## Key Concepts Demonstrated:
1. Multi-agent system (sequential agents)
2. Custom tools for notification management
3. Sessions & Memory (long-term memory storage)
4. Observability (logging and tracing)
5. Agent evaluation (LLM-as-judge)


In [ ]:
# Notification Data Structure and Memory Management
from datetime import datetime
from typing import Dict, List, Optional
from dataclasses import dataclass

@dataclass
class Notification:
    """Represents a notification from an app or service"""
    id: str
    app: str
    title: str
    body: str
    timestamp: str
    category: Optional[str] = None
    
    def to_dict(self) -> Dict:
        return {
            "id": self.id,
            "app": self.app,
            "title": self.title,
            "body": self.body,
            "timestamp": self.timestamp,
            "category": self.category
        }
    
    def __str__(self) -> str:
        return f"[{self.app}] {self.title}: {self.body}"

# Simple in-memory storage for demonstration
class NotificationMemory:
    """Simple memory store for less urgent notifications"""
    def __init__(self):
        self.memories: List[Dict] = []
    
    def store(self, notification: Notification, extracted_fact: str):
        """Store a notification fact in memory"""
        memory = {
            "notification_id": notification.id,
            "app": notification.app,
            "extracted_fact": extracted_fact,
            "timestamp": notification.timestamp,
            "stored_at": datetime.now().isoformat()
        }
        self.memories.append(memory)
        print(f"💾 Stored memory: {extracted_fact}")
        return memory
    
    def get_all(self) -> List[Dict]:
        """Retrieve all stored memories"""
        return self.memories
    
    def search(self, query: str) -> List[Dict]:
        """Simple keyword search (would use vector DB in production)"""
        query_lower = query.lower()
        return [
            m for m in self.memories
            if query_lower in m["extracted_fact"].lower() or 
               query_lower in m["app"].lower()
        ]

# Initialize memory store
memory_store = NotificationMemory()
print("✅ Notification and Memory classes initialized")


In [ ]:
# Custom Tools for Notification Management

def display_urgent_notification(app: str, title: str, body: str) -> dict:
    """
    Display an urgent notification to the user immediately.
    This tool is called when a notification requires immediate attention.
    
    Args:
        app: The name of the app sending the notification
        title: The notification title
        body: The notification body text
        
    Returns:
        Dictionary with status and result information.
        Success: {"status": "success", "message": "Notification displayed"}
        Error: {"status": "error", "error_message": "..."}
    """
    print(f"\n{'='*60}")
    print(f"🚨 URGENT NOTIFICATION")
    print(f"{'='*60}")
    print(f"App: {app}")
    print(f"Title: {title}")
    print(f"Body: {body}")
    print(f"{'='*60}\n")
    return {"status": "success", "message": f"Urgent notification from {app} displayed to user"}

def block_notification(app: str, title: str, reason: str) -> dict:
    """
    Block/suppress an irrelevant notification.
    This tool is called when a notification is determined to be noise.
    
    Args:
        app: The name of the app sending the notification
        title: The notification title
        reason: The reason for blocking (e.g., "social media noise", "promotional content")
        
    Returns:
        Dictionary with status and result information.
        Success: {"status": "success", "message": "Notification blocked"}
        Error: {"status": "error", "error_message": "..."}
    """
    print(f"🚫 Blocked: [{app}] {title} - {reason}")
    return {"status": "success", "message": f"Notification from {app} blocked: {reason}"}

def save_notification_memory(app: str, title: str, body: str, extracted_fact: str) -> dict:
    """
    Save a less urgent notification as a memory for later review.
    This tool extracts key information and stores it for future reference.
    
    Args:
        app: The name of the app sending the notification
        title: The notification title
        body: The notification body text
        extracted_fact: The key fact or information extracted from the notification
        
    Returns:
        Dictionary with status and result information.
        Success: {"status": "success", "message": "Memory stored"}
        Error: {"status": "error", "error_message": "..."}
    """
    # Create a temporary notification object for storage
    temp_notification = Notification(
        id=str(uuid.uuid4()),
        app=app,
        title=title,
        body=body,
        timestamp=datetime.now().isoformat()
    )
    memory_store.store(temp_notification, extracted_fact)
    return {"status": "success", "message": f"Memory stored: {extracted_fact}"}

print("✅ Custom tools defined")


In [ ]:
# Create the Focus Filter Agent
focus_filter_agent = LlmAgent(
    name="focus_filter_agent",
    model=Gemini(model="gemini-2.0-flash-exp", retry_options=retry_config),
    instruction="""You are Focus Filter, an intelligent notification filtering agent.

Your job is to analyze notifications and classify them into one of three categories:

1. **URGENT**: Requires immediate attention or action
   - Security alerts (bank, account access)
   - Critical deadlines or time-sensitive tasks
   - Emergency communications
   - Important personal messages requiring immediate response

2. **IRRELEVANT**: Noise that should be blocked
   - Social media likes, follows, generic updates
   - Marketing/promotional content
   - Low-value informational updates
   - Spam or unwanted notifications

3. **LESS URGENT**: Important but not immediate - should be stored in memory
   - Project updates, deadline changes
   - Informational updates worth remembering
   - Non-critical but useful information
   - Things the user might want to reference later

For each notification you receive:
1. Analyze the app, title, and body text
2. Classify it into one of the three categories
3. Use the appropriate tool:
   - Use `display_urgent_notification()` for URGENT items
   - Use `block_notification()` for IRRELEVANT items (provide a clear reason)
   - Use `save_notification_memory()` for LESS URGENT items (extract the key fact to remember)

Be conservative with URGENT - only use it for truly time-sensitive or critical items.
When saving to memory, extract the most important fact or information from the notification.""",
    tools=[
        display_urgent_notification,
        block_notification,
        save_notification_memory,
    ],
)

print("✅ Focus Filter agent created")
print("🔧 Available tools:")
print("  • display_urgent_notification - Shows critical alerts immediately")
print("  • block_notification - Suppresses irrelevant notifications")
print("  • save_notification_memory - Stores important information for later")


In [ ]:
# Create a runner and session service
session_service = InMemorySessionService()
runner = Runner(
    agent=focus_filter_agent,
    session_service=session_service,
)

print("✅ Runner and session service initialized")


## Testing the Agent

Let's test the agent with sample notifications:


In [ ]:
# Test notification 1: Urgent security alert
test1 = await runner.run(
    session_id="test_session_1",
    user_message="I received a notification: App: Banking App, Title: Security Alert, Body: Your bank flagged suspicious activity on your account. Please verify immediately."
)

print("\n" + "="*70)
print("Test 1 Complete")
print("="*70)


In [ ]:
# Test notification 2: Irrelevant social media
test2 = await runner.run(
    session_id="test_session_2",
    user_message="I received a notification: App: Social Media, Title: New Like, Body: 3 new people liked your photo."
)

print("\n" + "="*70)
print("Test 2 Complete")
print("="*70)


In [ ]:
# Test notification 3: Less urgent project update
test3 = await runner.run(
    session_id="test_session_3",
    user_message="I received a notification: App: Project Manager, Title: Deadline Update, Body: Your project deadline has moved to Tuesday."
)

print("\n" + "="*70)
print("Test 3 Complete")
print("="*70)


In [ ]:
# Display stored memories
print("\n" + "="*70)
print("💾 Stored Memories:")
print("="*70)

memories = memory_store.get_all()
if memories:
    for i, memory in enumerate(memories, 1):
        print(f"\n{i}. {memory['extracted_fact']}")
        print(f"   From: {memory['app']} (stored at {memory['stored_at']})")
else:
    print("No memories stored yet.")


## Architecture Overview

This implementation demonstrates:

1. **Agent-powered by LLM**: Uses Gemini 2.0 Flash for intelligent classification
2. **Custom Tools**: Three tools for notification management (display, block, save)
3. **Memory Management**: Simple in-memory storage (can be extended to vector DB)
4. **Observability**: Built-in ADK logging and tracing
5. **Agentic Loop**: Get Mission → Think → Act → Observe pattern

## Next Steps for Full Implementation

- Add vector database for semantic memory search
- Implement multi-agent architecture (separate agents for classification, action, memory)
- Add context engineering with few-shot examples
- Implement full observability with tracing
- Add agent evaluation framework with LLM-as-judge
- Add user preference learning
